In [1]:
import warnings
warnings.filterwarnings("ignore")

Searching and Downloading Data with Sunpy
=========================================

In this notebook we will look at how sunpy can be used to search and download solar physics data. Here is what this notebook contains:

1. Overview of Fido
2. Attributes of Fido
3. Constructing a data search query
4. Inspecting data search query
5. Using Fido to query the HEK
6. Downloading data
7. Extending Fido : New clients and use for your own packages <img src="https://raw.githubusercontent.com/sunpy/sunpy.org/main/_static/img/sunpy_icon.svg" alt="logo" width="100" align="right"/>


Fido Unified Downloader
====================

## 1. Overview of Fido



* Fido is sunpy's interface for searching and downloading solar physics data.


* It offers a unified interface for searching and fetching data irrespective of the underlying client or webservice from where the data is obtained.


* Offers a way to search and accesses multiple instruments and all available data providers in a single query.


* It supplies a single, easy, consistent and *extendable* way to get most forms of solar physics data the community need 

Fido offers access to data available through:

 * **VSO**
 * **JSOC** (through `drms`)
 * **Individual data providers** from web accessible sources (http, ftp, etc)
 * **HEK**
 * **HELIO**
 
Lets first import Fido and the attrs submodule, and inspect Fido. 
As described here Fido provides access to many sources of data through different `clients`, these clients can be defined inside sunpy or in other packages. Lets print the current list of available clients within sunpy.

In [2]:
from sunpy.net import Fido, attrs as a

In [3]:
Fido

Client,Description
EVEClient,Provides access to Level 0C Extreme ultraviolet Variability Experiment (EVE) data.
GBMClient,Provides access to data from the Gamma-Ray Burst Monitor (GBM) instrument on board the Fermi satellite.
XRSClient,Provides access to the GOES XRS fits files archive.
SUVIClient,Provides access to data from the GOES Solar Ultraviolet Imager (SUVI).
GONGClient,Provides access to the Magnetogram products of NSO-GONG synoptic Maps.
...,...
NoRHClient,Provides access to the Nobeyama RadioHeliograph (NoRH) averaged correlation time series data.
RHESSIClient,Provides access to the RHESSI observing summary time series data.
HEKClient,Provides access to the Heliophysics Event Knowledgebase (HEK).
HECClient,Provides access to the HELIO webservices.


## 2. Using attributes to search for data with Fido

Sunpy uses specified *attributes* to search for data using Fido. The range of these attributes is located in the `attrs` submodule. These `attr` parameters can be combined together to construct data search queries, such as searching over a certain time period, for data from a certain instrument with a certain wavelength etc.

Different clients and provides will have client-specific attributes, but the core attributes are:

* `a.Time`
* `a.Instrument`
* `a.Wavelength`


Lets look at how these attributes work in more detail

In [4]:
from astropy import units as u

In [5]:
print(a.Time("2021-05-23 02:00", "2021-05-23 04:00"))

<sunpy.net.attrs.Time(2021-05-23 02:00:00.000, 2021-05-23 04:00:00.000)>


We can inspect the instrument attribute to see what instrument `attrs` are currently supported through sunpy. Here we can see the instrument name (i.e. the name to be passed to the `a.Instrument` attribute, the client from which the data is available to access, and the full name of the instrument.

In [6]:
print(a.Instrument)

sunpy.net.attrs.Instrument

Specifies the Instrument name for the search.

       Attribute Name          Client          Full Name                                           Description                                   
--------------------------- ----------- ------------------------ --------------------------------------------------------------------------------
aia                         VSO         AIA                      Atmospheric Imaging Assembly                                                    
bbi                         VSO         BBI                      None                                                                            
bcs                         VSO         BCS                      Bragg Crystal Spectrometer                                                      
be_continuum                VSO         BE-Continuum             INAF-OACT Barra Equatoriale Continuum Instrument                                
be_halpha                   VSO         BE-Halpha

Sunpy also now provides tab completion to auto-fill the attribute name

In [9]:
a.Instrument.eit

<sunpy.net.attrs.Instrument(EIT: Extreme ultraviolet Imaging Telescope) object at 0x11167a5e0>

In [10]:
a.Wavelength(17.1*u.angstrom)

<sunpy.net.attrs.Wavelength(17.1, 17.1, 'Angstrom')>

## 3. Constructing a search query
 ### A simple query

Lets create a simple query to search for data from AIA over a particular time period

In [11]:
result = Fido.search(a.Time("2021-05-23 11:00", "2021-05-23 11:20"), 
                     a.Instrument("AIA"))

The `Fido.search` will return `UnifiedResponse` object with all the information available from the search that fit the criteria specified by the attrs. This `result` can then be passed to `Fido.fetch` to download the queried data (We will discuss this further below). Lets now inspect our `result` which holds the results of our query, if we print this we should see a summary of our query:

In [12]:
result

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info
,,,,Angstrom,,,,,,,Mibyte,
object,object,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str57
2021-05-23 11:00:00.000,2021-05-23 11:00:01.000,SDO,AIA,335.0 .. 335.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.901 exposure] [100.00 percentd]"
2021-05-23 11:00:04.000,2021-05-23 11:00:05.000,SDO,AIA,193.0 .. 193.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:00:05.000,2021-05-23 11:00:06.000,SDO,AIA,304.0 .. 304.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.902 exposure] [100.00 percentd]"
2021-05-23 11:00:05.000,2021-05-23 11:00:06.000,SDO,AIA,4500.0 .. 4500.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [0.300 exposure] [100.00 percentd]"
2021-05-23 11:00:06.000,2021-05-23 11:00:07.000,SDO,AIA,131.0 .. 131.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.901 exposure] [100.00 percentd]"
2021-05-23 11:00:09.000,2021-05-23 11:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:00:09.000,2021-05-23 11:00:10.000,SDO,AIA,211.0 .. 211.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.901 exposure] [100.00 percentd]"
2021-05-23 11:00:11.000,2021-05-23 11:00:12.000,SDO,AIA,94.0 .. 94.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.901 exposure] [100.00 percentd]"


Now lets make our query a bit more specific, say, say we only want one wavelength band from AIA. This can be achieved by specifying the `Wavelength` attribute within the search. The `Wavelength` attribute is passed as an astropy Quantity:

In [13]:
result = Fido.search(a.Time("2021-05-23 11:00", "2021-05-23 11:20"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(171*u.angstrom))

In [14]:
result

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info
,,,,Angstrom,,,,,,,Mibyte,
object,object,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str57
2021-05-23 11:00:09.000,2021-05-23 11:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:00:21.000,2021-05-23 11:00:22.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:00:33.000,2021-05-23 11:00:34.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:00:45.000,2021-05-23 11:00:46.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:00:57.000,2021-05-23 11:00:58.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:01:09.000,2021-05-23 11:01:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:01:21.000,2021-05-23 11:01:22.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:01:33.000,2021-05-23 11:01:34.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"


We can further specify this query by choosing the cadence (time-sampling) of the data we want to search for and download. This can be achieved by using the `Sample` attribute. Similar to the `Wavelength` attributes, this needs to be an astropy Quantity. Lets further specify the search above to only search for data with a cadence of 2 minutes.

In [15]:
result = Fido.search(a.Time("2021-05-23 11:00", "2021-05-23 11:20"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(171*u.angstrom),
                     a.Sample(2*u.min))

In [16]:
result

Start Time,End Time,Source,Instrument,Wavelength [2],Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size,Info
,,,,Angstrom,,,,,,,Mibyte,
object,object,str3,str3,float64,str4,str9,str6,str4,str4,str8,float64,str57
2021-05-23 11:00:09.000,2021-05-23 11:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:02:09.000,2021-05-23 11:02:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:04:09.000,2021-05-23 11:04:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:06:09.000,2021-05-23 11:06:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:08:09.000,2021-05-23 11:08:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:10:09.000,2021-05-23 11:10:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:12:09.000,2021-05-23 11:12:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"
2021-05-23 11:14:09.000,2021-05-23 11:14:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844,"AIA level 1, 4096x4096 [2.000 exposure] [100.00 percentd]"


In [17]:
result.file_num

10

###  More complex queries

In addition to making a query to one client for one instrument, Fido allows the flexibility to search for data from multiple instruments, wavelengths, times etc, even when the data is being obtained through different clients.

This query can be constructed by using the pipe `|` operator, which joins queries together just like the `OR` operator.

Lets now make a query that searches for both Nobeyama and Fermi/GBM data over a particular time period

In [18]:
result = Fido.search(a.Time("2013-10-28 01:00", "2013-10-28 03:00"), 
                     a.Instrument.gbm  | a.Instrument.norh)

In [19]:
result

Start Time,End Time,Instrument,Physobs,Source,Provider,Resolution,Detector
object,object,str3,str4,str5,str4,str5,str2
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n0
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n1
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n2
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n3
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n4
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n5
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n6
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n7
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n8


If we then want to further specify our search for each instrument, e.g. lets only search for detector n5 for GBM and only the 17GHz data from Nobeyama, this can be achieved by using the & operator 

In [20]:
result = Fido.search(a.Time("2013-10-28 01:00", "2013-10-28 03:00"), 
                     a.Instrument.gbm & a.Detector("n5") | a.Instrument.norh & a.Wavelength(17*u.GHz))

In [21]:
result

<sunpy.net.fido_factory.UnifiedResponse object at 0x1479687f0>
Results from 2 Providers:

2 Results from the GBMClient:
       Start Time               End Time        ... Resolution Detector
----------------------- ----------------------- ... ---------- --------
2013-10-28 00:00:00.000 2013-10-28 23:59:59.999 ...      cspec       n5
2013-10-28 00:00:00.000 2013-10-28 23:59:59.999 ...      ctime       n5

1 Results from the NoRHClient:
       Start Time               End Time        ... Provider Wavelength
                                                ...             GHz    
----------------------- ----------------------- ... -------- ----------
2013-10-28 00:00:00.000 2013-10-28 23:59:59.999 ...      NRO       17.0

## 4. Dealing with search results and the `UnifiedResponse`

So far we have shown how queries can be constructed and returned as a `UnifiedResponse` object. This object can then be passed to `fido.fetch` to download the data. However lets first inspect the `UnifiedResponse` object to get a better idea of what it consists of. 

From above, we can see that what is returned from a `Fido.search` is the `UnifiedResponse` object that consists of a one record (for a simple query) or multiple records depending on the search. These records (stored as tables) can be accessed by indexing the `UnifiedResponse` object. It acts both like a two dimensional array, where the first dimension is the response index and the second index is the row index, and a dictionary where you can index the responses by the name of the client. For example, lets look at our last query example that searched for data from Fermi/GBM and Nobeyama. The first index is the Fermi/GBM result table, and the second index is the Nobeyama one:



In [22]:
result = Fido.search(a.Time("2013-10-28 01:00", "2013-10-28 03:00"), 
                     a.Instrument.gbm  | a.Instrument.norh)

In [23]:
result[0]

Start Time,End Time,Instrument,Physobs,Source,Provider,Resolution,Detector
object,object,str3,str4,str5,str4,str5,str2
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n0
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n1
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n2
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n3
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n4
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n5
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n6
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n7
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n8


In [24]:
result[1]

Start Time,End Time,Instrument,Source,Provider,Wavelength
,,,,,GHz
object,object,str4,str4,str3,float64
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,NORH,NAOJ,NRO,17.0
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,NORH,NAOJ,NRO,34.0


These can also be accessed from a dictionary like way:

In [25]:
result["GBM"]

Start Time,End Time,Instrument,Physobs,Source,Provider,Resolution,Detector
object,object,str3,str4,str5,str4,str5,str2
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n0
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n1
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n2
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n3
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n4
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n5
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n6
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n7
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n8


In [26]:
result["NoRH"]

Start Time,End Time,Instrument,Source,Provider,Wavelength
,,,,,GHz
object,object,str4,str4,str3,float64
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,NORH,NAOJ,NRO,17.0
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,NORH,NAOJ,NRO,34.0


Similarly, as mentioned above the first index is the response index, and the second is the row, so we can pull out  the individual rows. For example, say we only want the first file from the Fermi/GBM data

In [27]:
result["GBM", 0]

Start Time,End Time,Instrument,Physobs,Source,Provider,Resolution,Detector,url
object,object,str3,str4,str5,str4,str5,str2,str101
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n0,https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n0_131028_v00.pha


Similarly, normal slices operations that work with other Python sequences will also work here (i.e. result[0, ::5] will return every fifth row etc)

In [28]:
result["GBM", 0::5]

Start Time,End Time,Instrument,Physobs,Source,Provider,Resolution,Detector
object,object,str3,str4,str5,str4,str5,str2
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n0
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,cspec,n5
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,ctime,n0
2013-10-28 00:00:00.000,2013-10-28 23:59:59.999,GBM,flux,FERMI,NASA,ctime,n5


### Inspecting the result tables

Each of these tables indexed from the `UnifiedResponse` object are `QueryResponseTable` objects which are just `astropy.table` objects that can easily interacted with (e.g. indexed, filtered etc).

This is particularly handy when the results that are searched for are from metadata tables such as the HEK! But lets first look at how we can interact with the results table, and filter etc. Lets first look at how we can play around with each table

In [31]:
result.all_colnames

['Detector',
 'End Time',
 'Instrument',
 'Physobs',
 'Provider',
 'Resolution',
 'Source',
 'Start Time',
 'url']

In [32]:
result[0].colnames

['Start Time',
 'End Time',
 'Instrument',
 'Physobs',
 'Source',
 'Provider',
 'Resolution',
 'Detector',
 'url']

In [33]:
result[0]["url"]

https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n0_131028_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n1_131028_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n2_131028_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n3_131028_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n4_131028_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n5_131028_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n6_131028_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n7_131028_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n8_131028_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_cspec_n9_131028_v00.pha
https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2013/10/28/current/glg_ctime_n0_131028_v00.pha


## 5. Metadata queries

As well as Fido providing an interface to search for data files that can be downloaded, Fido also allows you to query metadata. Currently Fido supports metadata searching from the HEK, HELIO and JSOC. 

Similar to what we have seen so far, the search results of these clients are a UnifiedResponse object which can then be indexed and the QueryResponse table accessed like an astropy table. Lets look at an example of how we can use Fido to query the HEK.

Lets query for the active regions defined by SWPC over the past month. This can be done by using the HEK client specific attributes `a.hek.attrs` 

In [34]:
result_hek = Fido.search(a.Time("2021-05-01", "2021-05-31"), 
                         a.hek.AR, a.hek.FRM.Name=="NOAA SWPC Observer")

In [35]:
result_hek

gs_thumburl,comment_count,hpc_bbox,meanphotoenergydensity,frm_humanflag,hgc_coord,ar_spotarearepruncert,event_coordsys,obs_levelnum,hpc_coord,event_npixels,gs_imageurl,ar_polarity,totalenergydensityunit,frm_paramset,hrc_coord,event_starttime,ar_mtwilsoncls,event_type,meantwistalpha,intensmin,obs_meanwavel,frm_url,bound_chaincode,totalphotoenergy,noposition,ar_spotarearepr,active,intensmax,frm_versionnumber,ar_pilcurvature,area_uncert,obs_dataprepurl,hpc_geom,hgc_bbox,intensmedian,chaincodetype,obs_channelid,ar_spotareareprunit,event_clippedspatial,ar_noaaclass,SOL_standard,event_avg_rating,eventtype,intensunit,hpc_boundcc,event_mapurl,frm_contact,meanvertcurrentdensity,ar_penumbracls,intensmean,bound_ccstartc1,frm_name,area_atdiskcenter,frm_identifier,obs_observatory,event_description,ar_spotarearaw,unsignedflux,boundbox_c2ur,magfluxunit,highshearareapercent,obs_firstprocessingdate,maxmagfieldstrengthunit,sharp_noaa_ars,boundbox_c2ll,frm_institute,hrc_bbox,refs_orig,meangradienthorz,gwill,ar_mcintoshcls,event_maskurl,bound_ccstartc2,ar_spotarearawunit,gs_movieurl,event_score,event_expires,hrc_boundcc,event_probability,intensvar,frm_daterun,currentdensityunit,highsheararea,event_coordunit,hpc_y,hpc_x,search_instrument,ar_numspots,kb_archivdate,kb_archivist,intenstotal,sum_overlap_scores,unsignedvertcurrent,hgs_boundcc,intensskew,obs_includesnrt,rasterscan,obs_wavelunit,kb_archivid,search_frm_name,meaninclinationgamma,currenthelicityunit,boundbox_c1ur,ar_noaanum,area_atdiskcenteruncert,boundbox_c1ll,absnetcurrenthelicity,meancurrenthelicity,event_importance_num_ratings,highshearareaunit,ar_compactnesscls,currentunit,event_testflag,maxmagfieldstrength,event_c2error,hrc_r,hgs_y,obs_title,hgs_x,hcr_checked,frm_specificid,totalphotoenergydensity,event_title,gwillunit,ar_sumnegsignedflux,obs_instrument,ar_neutrallength,event_c1error,revision,hpc_radius,meanshearangle,event_endtime,ar_spotarearawuncert,meanenergydensityunit,event_importance,meangradienttotal,event_coord2,event_coord3,event_coord1,search_observatory,ar_sumpossignedflux,area_raw,meangradientvert,concept,ar_axislength,event_pixelunit,hgc_boundcc,log_r_value,totalphotoenergyunit,hgc_x,hrc_a,hgc_y,gs_galleryid,twistunit,ar_lengthunit,hgs_coord,ar_zurichcls,bound_ccnsteps,intenskurt,event_clippedtemporal,gradientunit,rasterscantype,search_channelid,unsignedcurrenthelicity,hgs_bbox,area_unit,savncpp,obs_lastprocessingdate,refs [2]
str1,str1,str129,object,str4,str22,object,str12,object,str30,object,str1,object,str1,str3,str41,str19,str10,str2,object,object,float64,str3,str1,object,str5,object,str4,object,object,object,object,str1,str186,str120,object,str1,str7,str1,str1,str1,str30,object,str1,str1,str1,str1,str25,object,str1,object,object,str18,int64,str9,str7,str95,object,object,float64,str1,object,str1,str1,str1,float64,str36,str110,str1,object,object,str3,str1,object,str1,str1,str22,str1,str1,int64,object,str19,str1,object,str7,float64,float64,str3,int64,str19,str9,object,str21,object,str1,object,str1,str1,str2,str82,str4,object,str1,float64,int64,object,float64,object,object,str1,str1,str1,str1,str5,object,float64,float64,int64,str1,int64,str5,str1,object,str1,str1,object,str7,object,float64,str1,str19,object,str19,object,str1,object,object,int64,object,int64,str3,object,object,object,str13,object,str1,str1,object,str1,float64,float64,int64,str1,str1,str1,str14,str1,object,object,str1,str1,str1,str7,object,str95,str3,object,str1,object
,0,"POLYGON((886.656 -343.374396,886.85976 -344.474778,890.54124 -334.898334,890.33688 -333.792666,886.656 -343.374396))",None,true,POINT(-132.51421 -21),None,UTC-HGS-TOPO,None,POINT(888.64524 -339.139962),None,,None,,N/A,POINT(0.99930064886026 249.111291875901),2021-05-01T00:00:00,ALPHA,AR,None,None,5e-05,N/A,,None,false,None,true,None,None,None,None,,010300000001000000050000006891ED7C3FB58B400DFCA886FD7575C046D3D9C9E0B68B40C4EDD0B0988775C04C1AA37554D48B4028EE78935FEE74C0693524EEB1D28B406B2A8BC2AEDC74C06891ED7C3FB58B400DFCA886FD7575C0,"POL

As we can see the results from the HEK returns metadata with lots of columns, we can use the `.show()` to specify the columns we want to show. Lets just print all columns out first

In [36]:
result_hek.all_colnames

['SOL_standard',
 'absnetcurrenthelicity',
 'active',
 'ar_axislength',
 'ar_compactnesscls',
 'ar_lengthunit',
 'ar_mcintoshcls',
 'ar_mtwilsoncls',
 'ar_neutrallength',
 'ar_noaaclass',
 'ar_noaanum',
 'ar_numspots',
 'ar_penumbracls',
 'ar_pilcurvature',
 'ar_polarity',
 'ar_spotarearaw',
 'ar_spotarearawuncert',
 'ar_spotarearawunit',
 'ar_spotarearepr',
 'ar_spotarearepruncert',
 'ar_spotareareprunit',
 'ar_sumnegsignedflux',
 'ar_sumpossignedflux',
 'ar_zurichcls',
 'area_atdiskcenter',
 'area_atdiskcenteruncert',
 'area_raw',
 'area_uncert',
 'area_unit',
 'bound_ccnsteps',
 'bound_ccstartc1',
 'bound_ccstartc2',
 'bound_chaincode',
 'boundbox_c1ll',
 'boundbox_c1ur',
 'boundbox_c2ll',
 'boundbox_c2ur',
 'chaincodetype',
 'comment_count',
 'concept',
 'currentdensityunit',
 'currenthelicityunit',
 'currentunit',
 'event_avg_rating',
 'event_c1error',
 'event_c2error',
 'event_clippedspatial',
 'event_clippedtemporal',
 'event_coord1',
 'event_coord2',
 'event_coord3',
 'event_co

Now lets only print the time, NOAA active region number, active region area, number of spots, and the Hale class

In [37]:
result_hek.show("event_starttime", "ar_noaanum", "area_atdiskcenter", "ar_numspots", "ar_mtwilsoncls")

event_starttime,ar_noaanum,area_atdiskcenter,ar_numspots,ar_mtwilsoncls
str19,int64,int64,int64,str10
2021-05-01T00:00:00,12821,91310264,1,ALPHA
2021-05-01T00:00:00,12820,60873508,3,BETA
2021-05-01T00:00:00,12818,60873508,1,ALPHA
2021-05-02T00:00:00,12818,30436754,1,ALPHA
2021-05-08T00:00:00,12822,395677824,5,BETA
2021-05-09T00:00:00,12822,547861568,7,BETA
2021-05-10T00:00:00,12822,547861568,8,BETA-GAMMA
2021-05-11T00:00:00,12822,395677824,10,BETA-GAMMA
2021-05-11T00:00:00,12823,121747016,6,BETA


We can also index the results table like before, using a dictionary like indexing

In [38]:
interesting_columns=["event_starttime", "ar_noaanum", "area_atdiskcenter", "ar_numspots", "ar_mtwilsoncls"]

In [39]:
result_hek["HEK", interesting_columns]

event_starttime,ar_noaanum,area_atdiskcenter,ar_numspots,ar_mtwilsoncls
str19,int64,int64,int64,str10
2021-05-01T00:00:00,12821,91310264,1,ALPHA
2021-05-01T00:00:00,12820,60873508,3,BETA
2021-05-01T00:00:00,12818,60873508,1,ALPHA
2021-05-02T00:00:00,12818,30436754,1,ALPHA
2021-05-08T00:00:00,12822,395677824,5,BETA
2021-05-09T00:00:00,12822,547861568,7,BETA
2021-05-10T00:00:00,12822,547861568,8,BETA-GAMMA
2021-05-11T00:00:00,12822,395677824,10,BETA-GAMMA
2021-05-11T00:00:00,12823,121747016,6,BETA


#### Saving results

We can then save these metadata results to a file, for example lets write it to a csv file

In [40]:
result_hek["HEK", interesting_columns].write("active_regions_may2021.csv", format="csv")

## 6. Downloading the data

Now we can show how data that is queried above can be downloaded. Once the data you have searched for (and filtered etc) is constructed into a query using `Fido.search`, you can then easily download them using `Fido.fetch`.

The data is downloaded via asynchronous and parallel download streams (via `parfive`), and also allows for failed data downloads to be recognized so that files can be re-requested if not downloaded. 

Lets now look at how a `UnifiedResponse` from a `Fido.search` can be passed to `Fido.fetch` to download the data

In [41]:
result = Fido.search(a.Time("2021-05-23 11:00", "2021-05-23 11:20"), 
                     a.Instrument("AIA"), 
                     a.Wavelength(171*u.angstrom),
                     a.Sample(2*u.min))

In [42]:
files = Fido.fetch(result)

Files Downloaded:   0%|          | 0/10 [00:00<?, ?file/s]

These files are downloaded to a local location set in the sunpy.config.file, which by default is `~/sunpy/data/`. `Fido.fetch` returns a `parfile.Results` object which gives the path to where the files are downloaded to 

In [43]:
print(files)

['/Users/laurahayes/sunpy/data/aia_lev1_171a_2021_05_23t11_00_09_35z_image_lev1.fits', '/Users/laurahayes/sunpy/data/aia_lev1_171a_2021_05_23t11_04_09_35z_image_lev1.fits', '/Users/laurahayes/sunpy/data/aia_lev1_171a_2021_05_23t11_12_09_35z_image_lev1.fits', '/Users/laurahayes/sunpy/data/aia_lev1_171a_2021_05_23t11_06_09_35z_image_lev1.fits', '/Users/laurahayes/sunpy/data/aia_lev1_171a_2021_05_23t11_14_09_35z_image_lev1.fits', '/Users/laurahayes/sunpy/data/aia_lev1_171a_2021_05_23t11_18_09_35z_image_lev1.fits', '/Users/laurahayes/sunpy/data/aia_lev1_171a_2021_05_23t11_02_09_35z_image_lev1.fits', '/Users/laurahayes/sunpy/data/aia_lev1_171a_2021_05_23t11_16_09_35z_image_lev1.fits', '/Users/laurahayes/sunpy/data/aia_lev1_171a_2021_05_23t11_08_09_35z_image_lev1.fits', '/Users/laurahayes/sunpy/data/aia_lev1_171a_2021_05_23t11_10_09_35z_image_lev1.fits']


You can also define what directory you want the files to be saved to by passing the directory path to the `path` keyword in Fido.fetch. For example, I want to download these files to a local directory `/Users/laurahayes/event_analysis/`

In [44]:
files = Fido.fetch(result, path='/Users/laurahayes/event_analysis/')

Files Downloaded:   0%|          | 0/10 [00:00<?, ?file/s]

In [45]:
print(files)

['/Users/laurahayes/event_analysis/aia_lev1_171a_2021_05_23t11_02_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/aia_lev1_171a_2021_05_23t11_12_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/aia_lev1_171a_2021_05_23t11_18_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/aia_lev1_171a_2021_05_23t11_04_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/aia_lev1_171a_2021_05_23t11_16_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/aia_lev1_171a_2021_05_23t11_06_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/aia_lev1_171a_2021_05_23t11_00_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/aia_lev1_171a_2021_05_23t11_10_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/aia_lev1_171a_2021_05_23t11_08_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/aia_lev1_171a_2021_05_23t11_14_09_35z_image_lev1.fits']


You can also use the properties of the query to define the path to where to save the files, for example you can save them in a subdirectory named after the instrument of the queried files:

In [46]:
files = Fido.fetch(result, path='/Users/laurahayes/event_analysis/{instrument}/')

Files Downloaded:   0%|          | 0/10 [00:00<?, ?file/s]

In [47]:
print(files[0])

/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_04_09_35z_image_lev1.fits


### Download errors
The parfive.Results that is returned from `Fido.fetch` also contains information on the failed downloads and the error that is associated with them. These can be accessed by:

In [48]:
files.errors

[]

However for this example, there were no errors, but in the case that there was, this would list the errors. If there were failed downloads, the parfive.Results can be passed back into `Fido.fetch` to retry downloading the failed downloads e.g.

In [49]:
Fido.fetch(files)

Files Downloaded: 0file [00:00, ?file/s]

['/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_04_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_14_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_02_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_12_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_06_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_10_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_16_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_00_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_08_09_35z_image_lev1.fits', '/Users/laurahayes/event_analysis/AIA/aia_lev1_171a_2021_05_23t11_18_09_35z_image_lev1.fits']

Again as we had no failed file downloads this just returns to us the files downloaded.

## 7. How Fido can be extended 